In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
# cuda를 사용할 수 있으면 cuda를 쓰고 아니면 cpu를 사용한다
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters 지정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset를 가져온다
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader에 데이터를 저장해 학습에 사용한다
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn layers들을 생성한다. 
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU() # ReLU를 활성화함수로 사용한다

In [7]:
# Initialization. 정규분포에 따라 가중치들을 초기화한다
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [8]:
# model 생성. 한층씩 쌓아올리는 방식으로 모델을 만들었다.
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed. 다중분류에 많이 쓰이는 CrossEntropyLoss를 사용했다
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 최적화 방법으로 Adam을 사용했다

In [10]:
total_batch = len(data_loader) # 배치 개수. 60000/100 = 600개의 배치가 있다.
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        # 정전파
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        # 역전파 -> 그레디언트 -> 가중치 업데이트
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch # 에포크당 평균 loss를 구한다

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost)) # 에포크당 평균 로스 출력

print('Learning finished')

Epoch: 0001 cost = 128.894104004
Epoch: 0002 cost = 35.964019775
Epoch: 0003 cost = 23.049594879
Epoch: 0004 cost = 16.093503952
Epoch: 0005 cost = 11.621295929
Epoch: 0006 cost = 8.491368294
Epoch: 0007 cost = 6.468834400
Epoch: 0008 cost = 4.874640942
Epoch: 0009 cost = 3.618112803
Epoch: 0010 cost = 2.798370838
Epoch: 0011 cost = 2.111162663
Epoch: 0012 cost = 1.615930200
Epoch: 0013 cost = 1.270143867
Epoch: 0014 cost = 1.011346817
Epoch: 0015 cost = 0.863811731
Learning finished


In [11]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9455999732017517
Label:  9
Prediction:  9


/Users/minkyukim/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/minkyukim/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
